# Categorizing news

Download the starter guide Jupyter notebook [here](https://learn.aylien.com/download/news_api_python_starter_guide.ipynb)  
Read the documentation [here](https://docs.aylien.com/newsapi/#getting-started)

## 1. Configuring Your API Connection<a class="anchor" id="Configuring_Your_API_Connection"></a>
 If you don't have any credentials yet, you can sign up for a free trial [here](https://newsapi.aylien.com/signup).

In [1]:

import pandas as pd
import numpy as np
import hashlib

#for sql handling
import psycopg2
import sql
from sql import engine
from sql import get_data


print('Complete')

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 2


.env file found and working
Complete


## 2. Categorizing

**Bring data from DBeaver**

In [2]:
news = get_data("SELECT * FROM table")
news.head()

/opt/homebrew/Caskroom/miniconda/base/envs/sql-practice/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


**Categories**

In [ ]:

keywords = { 1: ['maintenance repair and overhaul', 'maintenance repair', 'overhaul', 'line maintenance', 'base maintenance', 'heavy check', 'a-check', 'c-check', 'd-check', 'part-145', '145', 'maintenance', 'repair'],            5: ['design organisation', 'design organization', 'part-21J', '21J'],
            6 : ['manufacturer', 'part manufacturer', 'part-21G', '21G'],
            7 : ['supplemental type certificate', 'STC'],
            8 : ['European (Union) Aviation Safety Agency', 'European Aviation Safety Agency', 'EASA', 'Federal Aviation Administration', 'FAA', 'Airworthiness Directive', 'AD', 'Type Certificate', 'TC'],
            9 : ['defense', 'military'],
            10: ['investor', 'investors', 'downgrades', 'downgrade', 'stock', 'stocks', 'accelerated depreciation', 'finance', 'financing', 'loan', 'bond', 'bonds', 'bargain purchase option', 'capitalize', 'capitalise', 'collateral', 'compound interest', 'credit', 'debtor', 'liquid', 'trade', 'acquisition', 'amortisation', 'balance sheet', 'bankruptcy', 'capital cost', 'capital structure', 'cash flow', 'chapter 11', 'credit rating', 'funding', 'hedge', 'letter of credit', 'liquidation', 'liquidity', 'partnering', 'earning ratio', 'purchase agreement', 'rating', 'wet lease', 'working capital', 'loss', 'market'],
            2: ['job', 'jobs', 'new headquarters', 'new headquarter', 'new office', 'new offices', 'new hub', 'hub', 'opening', 'positions', 'production sites', 'trained professionals', 'hiring', 'new hangar', 'builds team', 'building team', 'labor force'],
            3: ['sustainability', 'sustainable', 'renewable', 'environmental', 'environmental footprint', 'climate change', 'sustainable development', 'carbon dioxide emissions', 'carbon emissions', 'recycling', 'biofuels', ]
}

**Products**

In [ ]:
products = { 10: 'Landing gear',
            11: 'Engine',
            12: 'Completion',
            13: 'VIP completion',
            14: 'Component',
            15: 'Support',
            16: 'LRU'}

**Functions to save info in database**

In [ ]:
# function to upload categories in a new table

def save_categories(id, cat_id):
    
    try:
        connection = psycopg2.connect(user="postgres",
                                    password="]+Dr\DXm()`o=L:+",
                                    host="35.234.81.23",
                                    port="5432",
                                    database="postgres")
        cursor = connection.cursor()

        postgres_insert_query = f"INSERT INTO capstone.news_categories (news_id, cat_id) VALUES ('{id}', {cat_id})"
        print(postgres_insert_query)
        cursor.execute(postgres_insert_query)

        connection.commit()
        count = cursor.rowcount
        print(count, "Record inserted successfully into mobile table")

    except (Exception, psycopg2.Error) as error:
        print("Failed to insert record into mobile table", error)

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")


In [ ]:
# function to upload the product to a new table

def save_products(id, prod_id):
    
    try:
        connection = psycopg2.connect(user="postgres",
                                    password="]+Dr\DXm()`o=L:+",
                                    host="35.234.81.23",
                                    port="5432",
                                    database="postgres")
        cursor = connection.cursor()

        postgres_insert_query = f"INSERT INTO capstone.news_products (news_id, prod_id) VALUES ('{id}', {prod_id})"
        print(postgres_insert_query)
        cursor.execute(postgres_insert_query)

        connection.commit()
        count = cursor.rowcount
        # print(count, "Record inserted successfully into mobile table")

    except (Exception, psycopg2.Error) as error:
        print("Failed to insert record into mobile table", error)

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")


**Looking for keywords for categories & products**

In [ ]:

# define function to look up keywords in body

def search_keyword(competitors_df):
    for row in range(len(competitors_df)):
        for key in keywords:
            for value in keywords[key]:
                lower_body = (competitors_df['body_en'][row]).lower()
                index = lower_body.find(value)
                if index != -1:
                    print(competitors_df['title_en'][row], key, value)
                    save_categories(competitors_df['id'][row], key)
                    break
    return competitors_df


In [ ]:
# define function to look up product and services in body

def search_product(competitors_df):
    for row in range(len(competitors_df)):
        for key, value in products.items():
            print(key, value)
            lower_body = (competitors_df['body_en'][row]).lower()
            lower_product = value.lower()
            index = lower_body.find(lower_product)
            print(index)
            if index != -1:
                print(competitors_df['title_en'][row], key, value)
                save_products(competitors_df['id'][row], key)
                break
    return competitors_df


In [ ]:
# APPLY KEYWORD SEARCH & UPLOAD CATEGORIES TO NEW TABLE

search_keyword(news)

In [ ]:
# CHECK PRODUCTS AND UPLOAD NEW TABLE

search_product(news)

# Ranking news according to the score (M&A, JV, P)

In [ ]:
keywords_ranking = {
                        1: ['m&a', 'merge', 'acquisition', 'acquire'],
                        2: ['joint venture'],
                        3: ['partnership']
                        }

In [ ]:
# function to upload the product to a new table

def save_score(id, ranking_id, score, weight):
    
    try:
        connection = psycopg2.connect(user="postgres",
                                    password="]+Dr\DXm()`o=L:+",
                                    host="35.234.81.23",
                                    port="5432",
                                    database="postgres")
        cursor = connection.cursor()

        postgres_insert_query = f"INSERT INTO capstone.news_scores(news_id, ranking_id, score, weight) VALUES ('{id}', {ranking_id}, {score}, {weight})"
        print(postgres_insert_query)
        cursor.execute(postgres_insert_query)

        connection.commit()
        count = cursor.rowcount
        # print(count, "Record inserted successfully into mobile table")

    except (Exception, psycopg2.Error) as error:
        print("Failed to insert record into mobile table", error)

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")


In [ ]:
# define function to look up product and services in body

def rank_news(competitors_df):
    word_found = 0
    weight = 0
    article_score = 0
        
    for row in range(len(competitors_df)):
        lower_body = (competitors_df['body_en'][row]).lower()

        # keywords
        for key, word_list in keywords_ranking.items(): 
            
            word_found = 0
            weight = 0
            
            total = 0
            
            # keywords inside list        
            for word in word_list:                   
                index = lower_body.find(word)
                if index != -1:
                    word_found += 1
                    print(competitors_df['title_en'][row], key, word)
                    print(f'word found {word_found} of times')
                    
                    if key == 1:
                        article_score = 1.0
                        weight = article_score * word_found
                        print('score: ', article_score, 'weight: ', weight)
                    elif key == 2:
                        article_score = 0.7
                        weight = article_score * word_found
                        print('score: ', article_score, 'weight: ', weight)
                    elif key == 3:
                        article_score = 0.5
                        weight = article_score * word_found
                        print('score: ', article_score, 'weight: ', weight) 
                    else:
                        article_score = 0
                        weight = article_score * word_found
                        print('score: ', article_score, 'weight: ', weight)
                    
                    total = total + weight
                    
                    # print('INSERTED: ', competitors_df['id'][row], key, article_score, total)
                    save_score(competitors_df['id'][row], key, article_score, total)
    # return competitors_df


In [ ]:
rank_news(news)